### Git versioning

In [1]:
import git  #Requires Gitpython installed, to install do "$ pip install Gitpython" in your terminal
import pkg_resources
openfisca_france_location = pkg_resources.get_distribution('openfisca-france').location
repo = git.Repo(openfisca_france_location)
repo.git.status()

sha = repo.head.object.hexsha

print "---This notebook have been runed with--- \n","Openfisca-france :", repo.head.object.hexsha, '\n' 
print "Commit short SHA:", repo.git.rev_parse(sha, short=4)
print "Branch: ", repo.git.rev_parse('--abbrev-ref', "HEAD") #equivalent to: $git rev-parse --abbrev-ref HEAD
print "Date (y/m/d):", repo.git.show("-s", '--format=%ci', 'HEAD^')[:-5],"\n"  # equivalent to: $git show -s --format=%ci HEAD^

print "Openfisca-core :", git.Repo(pkg_resources.get_distribution('openfisca-core').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('openfisca-core').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]
print "Openfisca-france-data :", git.Repo(pkg_resources.get_distribution('openfisca-france-data').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('openfisca-france-data').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]
print "OpenFisca-Aggregates :", git.Repo(pkg_resources.get_distribution('OpenFisca-Aggregates').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('OpenFisca-Aggregates').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]
print "OpenFisca-Survey-Manager :", git.Repo(pkg_resources.get_distribution('OpenFisca-Survey-Manager').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('OpenFisca-Aggregates').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]

---This notebook have been runed with--- 
Openfisca-france : 03f19d8d1c056fec5562f592814f9526b3e6c0a2 

Commit short SHA: 03f19
Branch:  mensualisation_wip
Date (y/m/d): 2016-06-01 08:56:21  

Openfisca-core : 4076f591f4bcb571391b65a989e1603fdc4cb759  Date: 2016-02-24 16:11:57 
Openfisca-france-data : 4581a82fd1456cbe0bdd86ab1b57db6fef37c8cf  Date: 2016-05-31 15:46:50 
OpenFisca-Aggregates : fd210d8cfdf6ed68f173d9d5c3eab4a4302ed2b9  Date: 2016-04-04 11:35:03 
OpenFisca-Survey-Manager : 868512d24307aa75cfa7bbbf8420542960963966  Date: 2016-04-04 11:35:03 


### Imports

In [2]:
%matplotlib inline
from __future__ import division

from openfisca_core import periods
from openfisca_plugin_aggregates.tests.reform.test_aggregates_mensualized import create_survey_scenario
import pandas as pd

import matplotlib.pyplot as plt

import numpy as np
from numpy import (datetime64, logical_and as and_, logical_not as not_, logical_or as or_, logical_xor as xor_,
maximum as max_, minimum as min_, round)


#Faire des filtres pour Mahdi survey_scenario.create_data_frame_by_entity_key_plural

pd.set_option('expand_frame_repr', False)  #utile quand on utilise pandas en mode console
pd.set_option("display.max_rows",201)
pd.set_option("display.max_columns",201)

# Create Survey Scenario

In [3]:
period = periods.period(2009)

In [4]:
period.this_year.this_month

Period((u'month', Instant((2009, 1, 1)), 1))

In [5]:
survey_scenario = create_survey_scenario(2009)
survey_scenario.new_simulation()

weight_foyers = survey_scenario.simulation.calculate_add('weight_foyers', period = period).astype(float)
weight_menages = survey_scenario.simulation.calculate_add('wprm', period = period).astype(float)
weight_familles = survey_scenario.simulation.calculate_add('weight_familles', period = period).astype(float)
weight_individus = survey_scenario.simulation.calculate_add('weight_individus', period = period).astype(float)

In [6]:
simulation = survey_scenario.simulation
scenario = survey_scenario

Entity can be accessed from the object simulation either by its plural or singular name. # TODO: Why do we have that difference ? 

In [7]:
print simulation.entity_by_key_plural, "\n" + "--"*20 + "\n"
print simulation.entity_by_key_singular

{'foyers_fiscaux': <openfisca_core.reforms.FoyersFiscaux object at 0x118d79ad0>, 'individus': <openfisca_core.reforms.Individus object at 0x118d371d0>, 'familles': <openfisca_core.reforms.Familles object at 0x118d2bf50>, 'menages': <openfisca_core.reforms.Menages object at 0x118d2b0d0>} 
----------------------------------------

{'foyer_fiscal': <openfisca_core.reforms.FoyersFiscaux object at 0x118d79ad0>, 'individu': <openfisca_core.reforms.Individus object at 0x118d371d0>, 'famille': <openfisca_core.reforms.Familles object at 0x118d2bf50>, 'menage': <openfisca_core.reforms.Menages object at 0x118d2b0d0>}


Formulas are based on a specific entity : 

In [8]:
famille = simulation.entity_by_key_singular["famille"]
famille_entity = simulation.entity_by_key_singular["famille"]
famille_entity.column_by_name.keys()

[u'nb_par',
 u'maries',
 u'concub',
 u'isol',
 u'proprietaire_proche_famille',
 u'residence_dom',
 u'residence_guadeloupe',
 u'residence_martinique',
 u'residence_guyane',
 u'residence_reunion',
 u'residence_mayotte',
 u'psoc',
 u'pfam',
 u'mini',
 u'aides_logement',
 u'crds_pfam',
 u'biact',
 u'br_pf',
 u'al_pac',
 u'aide_logement_base_ressources_eval_forfaitaire',
 u'aide_logement_neutralisation_rsa',
 u'aide_logement_base_ressources_defaut',
 u'aide_logement_base_ressources',
 u'aide_logement_montant_brut',
 u'aide_logement_montant',
 u'alf',
 u'als_nonet',
 u'alset',
 u'als',
 u'apl',
 u'aide_logement_non_calculable',
 u'aide_logement',
 u'crds_logement',
 u'statut_occupation_famille',
 u'zone_apl_famille',
 u'bourse_college',
 u'bourse_lycee_points_de_charge',
 u'bourse_lycee_nombre_parts',
 u'bourse_lycee',
 u'aah_base_ressources',
 u'aah_non_calculable',
 u'br_mv',
 u'asi_aspa_nb_alloc',
 u'asi',
 u'aspa_couple',
 u'aspa',
 u'ass',
 u'ass_base_ressources',
 u'acs_montant',
 u'cm

Here is the list of all formulas contained in `famille` entity.

In [9]:
individu = simulation.entity_by_key_singular["individu"]
famille = simulation.entity_by_key_singular["famille"]
foyer = simulation.entity_by_key_singular["foyer_fiscal"]
menage = simulation.entity_by_key_singular["menage"]

## Roles

Individuals in entities can have different roles

In [10]:
print "famille: ", famille.roles_key
print "foyer:", foyer.roles_key
print "menage", menage.roles_key

famille:  ['parents', 'enfants']
foyer: ['declarants', 'personnes_a_charge']
menage ['personne_de_reference', 'conjoint', 'enfants', 'autres']


Those are label role, that indicate the status of each member of an entity, those are used to construct simulation and are more low level than end_users notions. 

There exist also effective role, that are a bit more precise on one status.

In [11]:
menage.label_by_role_key

{'autres': u'Autres',
 'conjoint': u'Conjoint',
 'enfants': u'Enfants',
 'personne_de_reference': u'Personne de r\xe9f\xe9rence'}

In [12]:
from openfisca_france.model.base import QUIFAM, QUIFOY, QUIMEN

In [13]:
print "Famille:"
for role in QUIFAM.itervars():
    print role
print "---"; print u"Ménage:"
for role in QUIMEN.itervars():
    print role
print "---"; print u"Foyers:"
for role in QUIFOY.itervars():
    print role

Famille:
('chef', 0)
('part', 1)
('enf1', 2)
('enf2', 3)
('enf3', 4)
('enf4', 5)
('enf5', 6)
('enf6', 7)
('enf7', 8)
('enf8', 9)
('enf9', 10)
---
Ménage:
('pref', 0)
('cref', 1)
('enf1', 2)
('enf2', 3)
('enf3', 4)
('enf4', 5)
('enf5', 6)
('enf6', 7)
('enf7', 8)
('enf8', 9)
('enf9', 10)
---
Foyers:
('vous', 0)
('conj', 1)
('pac1', 2)
('pac2', 3)
('pac3', 4)
('pac4', 5)
('pac5', 6)
('pac6', 7)
('pac7', 8)
('pac8', 9)
('pac9', 10)


In [14]:
simulation.entity_by_key_singular['famille']

# Repérer par role

## Repérer les chefs de familles

In [15]:
famille.role_for_person_variable_name
simulation.holder_by_name["quifam"].array == 0

array([ True, False,  True, ..., False, False, False], dtype=bool)

## Repérer les déclarants fiscaux

In [16]:
famille.role_for_person_variable_name
simulation.holder_by_name["quifoy"].array == 0

array([ True, False,  True, ..., False, False, False], dtype=bool)

## Repérer les personnes de références des ménages

In [17]:
famille.role_for_person_variable_name
simulation.holder_by_name["quimen"].array == 0

array([ True, False, False, ..., False, False, False], dtype=bool)

# Sélectionner par roles

## Recupérer les chefs de familles

In [18]:
chefs_de_famille = simulation.holder_by_name["quifam"].array == 0
part = simulation.holder_by_name["quifam"].array == 1
enfants = simulation.holder_by_name["quifam"].array >= 2

On peut par exemple regarder leurs age comparé a leur partenaire

In [19]:
print pd.Series(simulation.calculate("age"))[chefs_de_famille].mean()
print pd.Series(simulation.calculate("age"))[part].mean()
print pd.Series(simulation.calculate("age"))[enfants].mean()

50.4012042093
49.1393133766
9.25191081681


## Recupérer les déclarants fiscaux

In [20]:
declarant1 = simulation.holder_by_name["quifoy"].array == 0
declarant2 = simulation.holder_by_name["quifoy"].array == 1
enfants = simulation.holder_by_name["quifoy"].array >= 2

In [21]:
print pd.Series(simulation.calculate("age"))[declarant1].mean()
print pd.Series(simulation.calculate("age"))[declarant2].mean()
print pd.Series(simulation.calculate("age"))[enfants].mean()

49.5778468116
51.9249816976
9.71900628759


## Récupérer les chefs de menages

In [22]:
declarant1 = simulation.holder_by_name["quimen"].array == 0
declarant2 = simulation.holder_by_name["quimen"].array == 1
enfants = simulation.holder_by_name["quimen"].array >= 2

In [23]:
print pd.Series(simulation.calculate("age"))[declarant1].mean()
print pd.Series(simulation.calculate("age"))[declarant2].mean()
print pd.Series(simulation.calculate("age"))[enfants].mean()

52.8236494206
49.1401189227
13.3649181366


# Sum by entity

## Individuals to any entity

Let's say that we want to know how much a household (`Famille`) has earned in labor income ( which depend of `Individus`). We can then sum by entity.

In [24]:
salaire_net_holder = simulation.compute_add("salaire_net") #Computed by individus

In [25]:
salaire_net_holder.entity

In [41]:
entity_index_array = simulation.holder_by_name[famille.index_for_person_variable_name].array 

Gives the entity index array that associate an individual id (which is its position in the array), with its idfam

In [27]:
famille.index_for_person_variable_name

'idfam'

In [85]:
target_array = np.empty(famille.count)
target_array.fill(0)
for role in range(10):  #max cardinality of roles
    boolean_filter = simulation.holder_by_name[famille.role_for_person_variable_name].array == role
    
    target_array[entity_index_array[boolean_filter]] += salaire_net_holder.array[boolean_filter]

In [87]:
target_array[:10]

array([     0.        ,      0.        ,    212.00001526,      0.        ,
        16087.        ,  16956.        ,  22405.99804688,  20803.        ,
            0.        ,      0.        ])

In [30]:
salaire_net_holder.array[:10]

array([     0.        ,      0.        ,      0.        ,    212.00001526,
            0.        ,      0.        ,   3667.        ,  12420.        ,
            0.        ,  16956.        ], dtype=float32)

In [31]:
entity_index_array[:10]

array([0, 0, 1, 2, 3, 3, 4, 4, 4, 5], dtype=int32)

In [37]:
zip(salaire_net_holder.array[:20], entity_index_array[:20])

[(0.0, 0),
 (0.0, 0),
 (0.0, 1),
 (212.00002, 2),
 (0.0, 3),
 (0.0, 3),
 (3667.0, 4),
 (12420.0, 4),
 (0.0, 4),
 (16956.0, 5),
 (22405.998, 6),
 (0.0, 6),
 (0.0, 6),
 (0.0, 6),
 (0.0, 7),
 (20803.0, 7),
 (0.0, 8),
 (0.0, 9),
 (9471.0, 10),
 (2708.0, 10)]

In [39]:
target_array[target_array == 16087]

array([], dtype=float64)

In [40]:
zip(target_array[:30], range(30))

[(0.0, 0),
 (0.0, 1),
 (212.00001525878906, 2),
 (0.0, 3),
 (0.0, 4),
 (16956.0, 5),
 (0.0, 6),
 (20803.0, 7),
 (0.0, 8),
 (0.0, 9),
 (0.0, 10),
 (0.0, 11),
 (0.0, 12),
 (0.0, 13),
 (0.0, 14),
 (8907.0, 15),
 (0.0, 16),
 (0.0, 17),
 (30205.001953125, 18),
 (27331.994140625, 19),
 (0.0, 20),
 (0.0, 21),
 (24358.998046875, 22),
 (4275.0, 23),
 (0.0, 24),
 (15672.9990234375, 25),
 (0.0, 26),
 (0.0, 27),
 (25413.0, 28),
 (0.0, 29)]

In [32]:
print target_array.shape
print salaire_net_holder.array.shape
print entity_index_array.shape

(55306,)
(112359,)
(112359,)


On voit bien que `target_array` est la somme des `salaire_net_holder.array` elements appartenants à un même foyer.

### Whith a condition.

Let's say we want to keep only the wage of the head of the household or of the concubine

In [33]:

boolean_filter = simulation.holder_by_name["quifam"].array == 0
target_array = np.empty(famille.count)
target_array.fill(0)
target_array[entity_index_array[boolean_filter]] += salaire_net_holder.array[boolean_filter]

In [34]:
head_of_household = target_array
head_of_household_count = np.sum(boolean_filter)

In [35]:
head_average_income = (head_of_household.sum()/head_of_household_count)/12

In [36]:
head_average_income_w = ((head_of_household*weight_individus[boolean_filter]).sum()/(weight_individus[boolean_filter].sum()))/12

In [37]:

boolean_filter = simulation.holder_by_name["quifam"].array == 1
target_array = np.empty(famille.count)
target_array.fill(0)
target_array[entity_index_array[boolean_filter]] += salaire_net_holder.array[boolean_filter]

In [38]:
concubine = target_array
concubine_count = np.sum(boolean_filter)

In [39]:
concubine_count

28662

In [40]:
(concubine.sum()/concubine_count)/12

811.95076977513099

In [41]:
head_average_income_w

1076.5673021402527

## From non individual entity to an other entity

Let's try to put tax household income to household income.

Since different entities doesn't necessarily contain the same individuals. The trick is to attribute the entity amount to the head of that entity ( i.e head of entity means `simulation.holder_by_name[any_entity_holder_name].array == 0` ),

In [81]:
irpp_holder = simulation.compute("irpp")

In [82]:
irpp_holder.array.shape

(59874,)

In [83]:
irpp_holder.entity

In [84]:
irpp_holder.array[:20]

array([    0.        ,     0.        ,     0.        , -1378.82971191,
        -108.48883057,  -792.61254883, -9160.15039062, -2966.81005859,
           0.        ,     0.        ,   153.5       , -2335.95263672,
           0.        , -1309.25500488,     0.        , -2104.82763672,
           0.        ,   166.15258789, -4151.70703125, -3237.30004883], dtype=float32)

In [116]:
del persons

In [134]:

entity_index_array = simulation.holder_by_name["quifoy"].array

entity_index_array = simulation.holder_by_name["quifoy"].array
roles = None
entity = foyer


target_array = np.empty(simulation.persons.count)
target_array.fill(0)
entity_index_array = simulation.persons.simulation.holder_by_name[foyer.index_for_person_variable_name].array

if roles is None:
    roles = range(entity.roles_count)
for role in roles:
    boolean_filter = simulation.holder_by_name[entity.role_for_person_variable_name].array == role

    target_array[boolean_filter] = simulation.compute_add("irpp").array[entity_index_array[boolean_filter]]

    
target_array[50]

-433.72232055664062

In [135]:
index_for_person_variable_name = famille.index_for_person_variable_name

In [159]:

def cast_from_entity_to_role( var , entity_key, roles = None):
    
    entity = simulation.entity_by_key_singular[entity_key]
    index_for_person_variable_name = entity.index_for_person_variable_name
    
    entity_index_array = simulation.holder_by_name[entity.index_for_person_variable_name].array
    
    if roles is None:
        roles = range(entity.roles_count)

        
    target_array = np.empty(simulation.persons.count)
    target_array.fill(0)
    for role in roles:
        boolean_filter = simulation.holder_by_name[entity.role_for_person_variable_name].array == role
        
        print boolean_filter


        target_array[boolean_filter] = simulation.compute_add(var).array[entity_index_array[boolean_filter]]
        
        print entity_index_array[boolean_filter][::10]
    return target_array

    



In [147]:
simulation.holder_by_name[foyer.index_for_person_variable_name].array[110:150]

array([53, 53, 54, 54, 55, 56, 56, 57, 57, 57, 58, 58, 59, 59, 60, 60, 61,
       61, 62, 62, 63, 64, 64, 65, 66, 67, 67, 68, 67, 69, 70, 71, 72, 73,
       74, 75, 76, 76, 77, 78], dtype=int32)

In [108]:
from openfisca_utils import manipulate_entities

In [109]:
manipulate_entities.sum_by_entity("irpp", 'famille')

NameError: global name 'simulation' is not defined

## Et après on somme sur la famille

In [107]:
sum_array_by_entity(target_array, 'famille')[:50]

array([    0.        ,     0.        ,     0.        , -1378.82971191,
        -108.48883057,  -792.61254883, -9160.15039062, -2966.81005859,
           0.        ,     0.        ,   153.5       , -2335.95263672,
           0.        , -1309.25500488,     0.        , -2104.82763672,
           0.        ,   166.15258789, -4151.70703125, -3237.30004883,
       -2124.67504883,  -877.31494141,  -601.9440918 ,   737.79998779,
           0.        , -1072.57531738, -1089.63220215,  -433.72232056,
           0.        ,     0.        ,     0.        , -1132.75500488,
       -1087.48754883,     0.        ,     0.        ,   778.0402832 ,
         -79.07733154, -5260.95019531,  -576.65887451, -1171.85229492,
        -157.12251282, -1710.03051758, -8168.14990234,  -502.41864014,
       -7868.875     , -7178.47509766, -7609.375     ,     0.        ,
        -700.63287354,     0.        ])

In [104]:
weight_foyers.shape

(59874,)

In [166]:
sum_array_by_entity(target_array, 'famille').shape

(55306,)

In [77]:
target_array2 = np.empty(famille.count)
target_array2.fill(0)

boolean_filter = simulation.holder_by_name["quifam"].array == 0

target_array2[entity_index_array[boolean_filter]] += target_array[boolean_filter]

In [79]:
target_array2[:20]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [165]:
famille.count

55306

In [167]:

def cast_from_entity_to_role( var , entity_key, roles = None):
    
    entity = simulation.entity_by_key_singular[entity_key]
    index_for_person_variable_name = entity.index_for_person_variable_name
    
    entity_index_array = simulation.holder_by_name[entity.index_for_person_variable_name].array
    
    if roles is None:
        roles = range(entity.roles_count)

        
    target_array = np.empty(simulation.persons.count)
    target_array.fill(0)
    for role in roles:
        boolean_filter = simulation.holder_by_name[entity.role_for_person_variable_name].array == role
        
        print boolean_filter


        target_array[boolean_filter] = simulation.compute_add(var).array[entity_index_array[boolean_filter]]
        
        print entity_index_array[boolean_filter][::10]
    return target_array

    



In [170]:
from openfisca_core import formulas

In [ ]:
formulas.SimpleFormula.cast_from_entity_to_role()

In [179]:
class EmptyClass:
    pass

In [180]:
empty_attribute = EmptyClass

In [181]:
empty_attribute.holder = simulation.compute("irpp")

In [183]:
formulas.SimpleFormula.sum_by_entity(empty_attribute)

TypeError: unbound method sum_by_entity() must be called with SimpleFormula instance as first argument (got classobj instance instead)

In [ ]:
simulation.holder_by_name["quifam"].array == 0

## Fonctions génériques :

In [92]:
def sum_by_entity(var, entity_key, boolean_filter = None):
    var_holder = simulation.compute_add(var)
    entity = simulation.entity_by_key_singular[entity_key]
    
    entity_index_array = simulation.holder_by_name[entity.index_for_person_variable_name].array
    
    if boolean_filter == None:
        boolean_filter = np.ones(simulation.persons.count, dtype=bool)
    else:
        boolean_filter
    
    

    target_array = np.empty(entity.count)
    target_array.fill(0)
    target_array[entity_index_array[boolean_filter]] += var_holder.array[boolean_filter]



    
    return target_array
    

In [93]:
sum_by_entity("salaire_net", 'foyer_fiscal', simulation.holder_by_name["quifoy"].array == 0)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


array([    0.        ,     0.        ,   212.00001526, ...,  4790.        ,
        1008.        ,     0.        ])

In [95]:
def sum_array_by_entity(array, entity_key, boolean_filter = None):

    entity = simulation.entity_by_key_singular[entity_key]
    
    entity_index_array = simulation.holder_by_name[entity.index_for_person_variable_name].array
    
    if boolean_filter == None:
        boolean_filter = np.ones(simulation.persons.count, dtype=bool)
    else:
        boolean_filter
    
    

    target_array = np.empty(entity.count)
    target_array.fill(0)
    target_array[entity_index_array[boolean_filter]] += array[boolean_filter]



    
    return target_array
    

In [217]:

def cast_from_entity_to_role( var , entity_key, roles = None):
    
    entity = simulation.entity_by_key_singular[entity_key]
    index_for_person_variable_name = entity.index_for_person_variable_name
    entity_index_array = simulation.holder_by_name[entity.index_for_person_variable_name].array
    
    if roles is None:
        roles = range(entity.roles_count)

        
    target_array = np.empty(simulation.persons.count)
    target_array.fill(0)
    for role in roles:
        boolean_filter = simulation.holder_by_name[entity.role_for_person_variable_name].array == role

        target_array[boolean_filter] = simulation.compute_add(var).array[entity_index_array[boolean_filter]]

    return target_array

    



In [234]:

def cast_from_entity_to_role( var , roles = None):
    
    var_holder = simulation.compute_add(var)
    entity = var_holder.entity
    entity_index_array = simulation.holder_by_name[entity.index_for_person_variable_name].array
    
    if roles is None:
        roles = range(entity.roles_count)

        
    target_array = np.empty(simulation.persons.count)
    target_array.fill(0)
    for role in roles:
        boolean_filter = simulation.holder_by_name[entity.role_for_person_variable_name].array == role

        target_array[boolean_filter] = simulation.compute_add(var).array[entity_index_array[boolean_filter]]

    return target_array


In [236]:
cast_from_entity_to_role('irpp').shape

(112359,)

In [232]:
impo = simulation.compute('irpp') ; impo.entity.index_for_person_variable_name

'idfoy'

In [224]:
simulation.entity_by_key_singular['foyer_fiscal']

In [197]:
def switch_entity(var, target_entity  ,roles = [0]):
    var_holder = simulation.compute_add(var)
    entity = var_holder.entity
    
    target_array = cast_from_entity_to_role(var, entity.key_singular, roles = roles)
    

    target_array = sum_array_by_entity(target_array, entity_key = target_entity )
    
    return target_array
    
    
    
    
    
    

In [198]:
switch_entity('irpp', 'famille')

[ True False  True ..., False False False]
[    0    10    20 ..., 59850 59860 59870]


array([     0.        ,      0.        ,      0.        , ...,
       -12667.37304688,      0.        ,      0.        ])

In [192]:
%debug

> <ipython-input-190-815b102ab333>(5)switch_entity()
      4 
----> 5     target_array = cast_from_entity_to_role(var, entity.key, roles = roles)
      6 

ipdb> entity
ipdb> type(entity)
<class 'openfisca_core.reforms.FoyersFiscaux'>
ipdb> q


In [184]:
simulation.compute("irpp")

In [189]:
irpp_holder.

In [ ]:
person_variable_holder = simulation.compute_add("salaire_net")
person_variable_holder.array.shape

In [ ]:
entity = simulation.entity_by_key_singular["famille"]
entity.index_for_person_variable_name

In [ ]:
simulation.holder_by_name[variable_holder.entity.index_for_person_variable_name].array

In [ ]:
sum_by_entity("salaire_net", 'famille')

In [ ]:
simulation.entity_by_key_singular[entity]